

# Example: Train SimCLR on CIFAR10
<a href="https://colab.research.google.com/github/melhaud/proj18/blob/main/examples/simclr-on-cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

In this tutorial, we will train a SimCLR model using lightly. The model,
augmentations and training procedure is from 
`A Simple Framework for Contrastive Learning of Visual Representations <https://arxiv.org/abs/2002.05709>`_.

The paper explores a rather simple training procedure for contrastive learning.


## Imports

Import the Python frameworks we need for this tutorial.



In [1]:
import os
import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from lightly.data import LightlyDataset
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Subset
import sys


sys.path.append('../src')
from utils import custom_collate_fn, get_classes
from my_resnet import resnet20
%matplotlib inline

## Configuration

We set some configuration parameters for our experiment.
Feel free to change them and analyze the effect.



In [2]:
num_workers = 2
batch_size = 128
seed = 1

max_epochs = 150
input_size = 32 # image height, assume its always square

# Let's set the seed for our experiments

pl.seed_everything(seed)

Global seed set to 1


1

## Setup data augmentations and loaders

The images from the dataset have been taken from above when the clothing was 
on a table, bed or floor. Therefore, we can make use of additional augmentations
such as vertical flip or random rotation (90 degrees). 
By adding these augmentations we learn our model invariance regarding the 
orientation of the clothing piece. E.g. we don't care if a shirt is upside down
but more about the strcture which make it a shirt.

You can learn more about the different augmentations and learned invariances
here: `lightly-advanced`.



In [3]:
cifar10_train = CIFAR10("../data/cifar10", download=True, train=True)
cifar10_test = CIFAR10("../data/cifar10", download=True, train=False)

classes_ids_train = get_classes(cifar10_train) # long!
classes_ids_test = get_classes(cifar10_test)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
collate_fn = lightly.data.SimCLRCollateFunction(
    input_size=input_size,
    vf_prob=0.5,
    rr_prob=0.5
)

# We create a torchvision transformation for embedding the dataset after 
# training
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((input_size, input_size)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.RandomRotation(degrees=(-10, 10)),
    torchvision.transforms.ToTensor(),
])

dataset_train_simclr = LightlyDataset.from_torch_dataset(Subset(cifar10_train, classes_ids_train['dog']))

dataloader_train_simclr = torch.utils.data.DataLoader(
    dataset_train_simclr,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers
)


dataset_test = LightlyDataset.from_torch_dataset(Subset(cifar10_train, classes_ids_train['dog']))

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers  cd
)

## Create the SimCLR Model
Now we create the SimCLR model. We implement it as a PyTorch Lightning Module
and use custom ResNet-20 backbone provided by Nikita Balabin. Lightly provides implementations
of the SimCLR projection head and loss function in the `SimCLRProjectionHead`
and `NTXentLoss` classes. We can simply import them and combine the building
blocks in the module. We will import constructed model from our `src`.



In [5]:
from simclr_model import SimCLRModel

We first check if a GPU is available and then train the module
using the PyTorch Lightning Trainer.



In [6]:
gpus = 1 if torch.cuda.is_available() else 0

resnet_backbone = resnet20(num_classes=1)
model = SimCLRModel(resnet_backbone, img_size = input_size)
trainer = pl.Trainer(
    max_epochs=50, gpus=gpus, progress_bar_refresh_rate=10
)
trainer.fit(model, dataloader_train_simclr)

/trinity/home/d.volkov/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                 | Params
---------------------------------------------------------
0 | backbone        | Sequential           | 269 K 
1 | projection_head | SimCLRProjectionHead | 6.2 K 
2 | criterion       | NTXentLoss           | 0     
---

Next we create a helper function to generate embeddings
from our test images using the model we just trained.
Note that only the backbone is needed to generate embeddings,
the projection head is only required for the training.
Make sure to put the model into eval mode for this part!



# Generate embeddings for test

In [8]:
from utils import generate_embeddings

model.eval()
embeddings_test, filenames_test = generate_embeddings(model, dataloader_test)
embeddings_train, filenames_train = generate_embeddings(model, dataloader_train_simclr)

In [9]:
print(f'Shape of TEST embeddings {embeddings_test.shape}')
print(f'Shape of TRAIN embeddings {embeddings_train.shape}')

Shape of TEST embeddings (5000, 64)
Shape of TRAIN embeddings (4992, 64)


# Calculate Hausdorff distance between point clouds

In [26]:
from scipy.spatial.distance import directed_hausdorff

hausdorff_dist = directed_hausdorff(embeddings_train, embeddings_test)[0]

print(f'Hausdorff Dist: {hausdorff_dist:.3f}')

Hausdorff Dist: 0.359
